In [80]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
df=pd.read_csv(r'C:\Users\Divyanshu Verma\OneDrive\Documents\Customer_Behaviour.csv')

In [35]:
df.head(4)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2.103810e+18,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2.053010e+18,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2.053010e+18,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2.053010e+18,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713


In [36]:
df.shape

(101773, 9)

In [37]:
df.isnull().sum()

event_time           0
event_type           0
product_id           0
category_id          0
category_code    33145
brand            14671
price                0
user_id              0
user_session         0
dtype: int64

In [38]:
df.dropna(inplace=True)

In [39]:
df.shape

(63095, 9)

In [40]:
df.describe()

,product_id,category_id,price,user_id
count,6.309500e+04,6.309500e+04,63095.000000,6.309500e+04
mean,5.321984e+06,2.055221e+18,355.335001,5.314347e+08
std,8.030400e+06,1.379550e+16,396.472246,1.664948e+07
min,1.001588e+06,2.053010e+18,0.880000,3.064418e+08
25%,1.004840e+06,2.053010e+18,100.380000,5.157633e+08
50%,1.600548e+06,2.053010e+18,203.350000,5.275800e+08
75%,5.100719e+06,2.053010e+18,462.250000,5.484495e+08
max,5.290010e+07,2.172370e+18,2574.070000,5.554840e+08


In [41]:
#dropping unwanted column,

df.drop(['event_time','product_id','category_id','user_session','price','user_id','event_type'],axis=1,inplace=True)


## I am using category_code as a tag based on brand

In [42]:
df.head(3)

,category_code,brand
1,appliances.environment.water_heater,aqua
3,computers.notebook,lenovo
4,electronics.smartphone,apple


In [43]:
df.shape

(63095, 2)

In [44]:
#Replacing special Character bcoz they can create isssue in recommendation .

df['category_code']=df['category_code'].apply(lambda x:x.replace('.',' ').replace('_',''))

In [45]:
df.head(2)

,category_code,brand
1,appliances environment waterheater,aqua
3,computers notebook,lenovo


In [47]:
df['brand'].nunique()

856

## Here I am removing duplicate values in brand column bcoz we dont want same category product will get recommended too many times.


In [52]:
final_df=df.drop_duplicates('brand')

In [56]:
final_df.shape

(856, 2)

In [67]:
# after removing duplicates values indexes of the data will be mismatched. since for our model index no. should be in ascending order without gap.
# this is the best technique to manage index issue

final_df.to_csv('Untitled888.csv', index=False)

In [68]:
final_df=pd.read_csv('Untitled888.csv')

In [70]:
#now you can see indexes are like 0,1,2 in ascending order without gap.
final_df.head(3)

,category_code,brand
0,appliances environment waterheater,aqua
1,computers notebook,lenovo
2,electronics smartphone,apple


In [71]:
#model
cv= CountVectorizer(max_features=500)

In [72]:
cv

CountVectorizer(max_features=500)

In [73]:
#Training the data

vectors=cv.fit_transform(final_df['category_code']).toarray()

In [74]:
vectors

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

## In the cosine similarity calculation, each row in the category code column will be compared with every other row to compute a distance. For instance, if you're comparing "electronics smartphone," with ''electronics smartphone. the cosine similarity score will be  1 because both two words  are matching well. On the other hand, if you're comparing "electronics smartphone'' with " electronic heater" in the category column, the score might be around 0.5 due to that half words are matching. The items with a score of 1 will be ranked higher, followed by items with a score around 0.7, and so on, all in descending order.

In [75]:
from sklearn.metrics.pairwise import cosine_similarity

In [76]:
similarity=cosine_similarity(vectors)

### You can see the score in this array

In [77]:
similarity[1]

array([0.        , 1.        , 0.        , 0.5       , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.40824829, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.40824829, 0.        ,
       0.5       , 0.        , 0.        , 0.        , 0.        ,
       0.5       , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.5       , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [78]:
#final function to recommend.

def recommend(brand):
    company_name=final_df[final_df['brand'] == brand].index[0]
    distances=similarity[company_name]
    name_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:40]
    
    for i in name_list:
        print(final_df.iloc[i[0]].brand)

### Here You Can see I Seached a Smartphone and in return its recommending me all smatphones and after that smart watches  bcoz smart watch's category code is ''electronics wacthes'' so the distance is around 0.5.

In [81]:

recommend('samsung')

huawei
samsung
xiaomi
nokia
oneplus
oppo
bq
fly
meizu
vivo
doogee
tp-link
jinga
gionee
google
blackberry
tecno
nubia
ark
zte
casio
garmin
prestigio
wacom
tissot
honor
microsoft
dkny
longines
aimoto
wonlex
orient
maxvi
fossil
armani
polar
canyon
withings
rado


In [83]:
pip install pandoc

Note: you may need to restart the kernel to use updated packages.
